In [1]:
#!python -m pip install -U pytorch

In [105]:
#!python -m pip install -U transformers
#from sagemaker.pytorch import PyTorch
from sagemaker.pytorch import PyTorch
from transformers import AutoModelForSequenceClassification,AutoTokenizer,pipeline

model_dianp = AutoModelForSequenceClassification.from_pretrained('uer/roberta-base-finetuned-dianping-chinese')
tokenizer_dianp = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-dianping-chinese')


In [115]:
import torch
torch.cuda.device_count()

1

In [106]:
model_jd = AutoModelForSequenceClassification.from_pretrained('uer/roberta-base-finetuned-jd-full-chinese')
tokenizer_jd = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-jd-full-chinese')


In [107]:
text_classification = pipeline('sentiment-analysis', model=model_dianp, tokenizer=tokenizer_dianp)



In [108]:
import pandas as pd
dfxq = pd.read_excel('sample_posts_20220324.xlsx', '600196_from_xueqiu', engine='openpyxl', )


In [109]:
dfgb = pd.read_excel('sample_posts_20220324.xlsx', '601607_from_guba', engine = 'openpyxl')


In [110]:
dfgb.head(1)

,ticker,ticker_name,url,post_id,post_title,post_abstract,post_content
0,601607,上海医药,"https://guba.eastmoney.com/news,601607,9261090...",926109040,上药垃圾，都不要买,上药垃圾，都不要买,上药垃圾，都不要买


In [111]:
dfgb = dfgb[dfgb['post_content'].notna()]

###文本清洗（清洗后的column名字为clean_cmt）
import re
def process_text(text):
    text = re.sub(r"[0-9]+", '', text) 
    text = re.sub(r'\s+',' ',text)
    text = text.lower()
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    text = ''.join(e for e in text if e.isalnum())
    return text

content = dfgb['post_content'].dropna() #把空的content去掉
processed_content = []
for i in range(len(content)):
    processed_content.append([process_text(content.iloc[i])])

In [112]:
import numpy as np
dfgb["clean_cmt"] = np.array(processed_content)

In [113]:
ratings = []
i = 0
for note in processed_content:
    note = str(note)
    print(i)
    i = i+1 
    if len(note.encode('utf-8').decode('utf-8')) > 511:  #对于超过512个字符的评论，只得进行截取前512个字符
        note = note.encode('utf-8').decode('utf-8')[:510]
        try:
            ratings.append(text_classification(note))
        except Exception as e:
            print(e, note)
            pass
    else:
        ratings.append(text_classification(note))
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [102]:
dfgb["ratings"] = ratings

In [103]:
dfgb.head()

,ticker,ticker_name,url,post_id,post_title,post_abstract,post_content,clean_cmt,ratings
0,601607,上海医药,"https://guba.eastmoney.com/news,601607,9261090...",926109040,上药垃圾，都不要买,上药垃圾，都不要买,上药垃圾，都不要买,上药垃圾都不要买,"[{'label': 'negative (stars 1, 2 and 3)', 'sco..."
1,601607,上海医药,"https://guba.eastmoney.com/news,601607,9408257...",940825794,谁在发上海医药利好！明天跌停,谁在发上海医药利好！明天跌停,谁在发上海医药利好！明天跌停,谁在发上海医药利好明天跌停,"[{'label': 'negative (stars 1, 2 and 3)', 'sco..."
2,601607,上海医药,"https://guba.eastmoney.com/news,601607,9427955...",942795507,完了完了,垃圾上涨时就是大盘见顶时。几乎所有板块都在狂欢，这里却在耗尽个股牛市的最后时间。,垃圾上涨时就是大盘见顶时。几乎所有板块都在狂欢，这里却在耗尽个股牛市的最后时间。,垃圾上涨时就是大盘见顶时几乎所有板块都在狂欢这里却在耗尽个股牛市的最后时间,"[{'label': 'negative (stars 1, 2 and 3)', 'sco..."
3,601607,上海医药,"https://guba.eastmoney.com/news,601607,9441497...",944149709,情况不妙呀走为上策,情况不妙呀走为上策,情况不妙呀走为上策,情况不妙呀走为上策,"[{'label': 'negative (stars 1, 2 and 3)', 'sco..."
4,601607,上海医药,"https://guba.eastmoney.com/news,601607,9490408...",949040885,行到水穷处，坐看云起时！,行到水穷处，坐看云起时！,行到水穷处，坐看云起时！,行到水穷处坐看云起时,"[{'label': 'negative (stars 1, 2 and 3)', 'sco..."


In [104]:
dfgb.to_excel('gb_roberta_dianp_ratings.xlsx')

In [89]:
text_classification("股票交易终极的本质就是资本的趋利避害股票交易终极的本质就是资本的趋利避害。$上海机场(SH600009)$ $长春高新(SZ000661)$ $复星医药(SH600196)$")


[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.7818922996520996}]

In [90]:
text_classification("股票交易终极的本质就是资本的趋利避害")


[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.645952582359314}]

In [91]:
text_classification("毫不夸张地说，当下的经济情况，距离经济滞胀已经不远了。如果不是相对宽松的货币政策，仍然在推动着整体经济的前进，经济增长的速度可能会更慢。而广义货币M2的增速也比GDP的增长来的更快，从这个角度来看")


[{'label': 'positive (stars 4 and 5)', 'score': 0.5665899515151978}]

In [92]:
text_classification("毫不夸张地说，当下的经济情况，距离经济滞胀已经不远了。 如果不是相对宽松的货币政策，仍然在推动着整体经济的前进，经济增长的速度可能会更慢。 而广义货币M2的增速也比GDP的增长来的更快，从这个角度来看，钱的购买力能不能保值，还有待商榷。 其实，这是一个正常的现象，在全球范围内都面临...")






[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.7222153544425964}]